In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.integrate import quad
from scipy.interpolate import interp1d

# Define a function to concatenate multiple columns into a single column
def concatenate_columns(df):
    return df.values.flatten()

# Compute the 2/5 power integral of the second derivative after Savitzky-Golay filtering
def compute_second_derivative_integral(x, y):
    if np.any(np.diff(x) <= 0):
        print("Warning: x is not strictly increasing, skipping second derivative computation")
        return np.nan
    # Compute second derivative using Savitzky-Golay filter
    y_savgol_second_derivative = savgol_filter(y, window_length=min(31, len(y) - 1), polyorder=3, deriv=2)
    # Compute the integral of the 2/5 power
    integral, _ = quad(lambda eps: np.interp(eps, x, np.abs(y_savgol_second_derivative) ** (2/5)), np.min(x), np.max(x))
    return integral ** (5/2)

# Compute the optimal number of segments K
def compute_optimal_segments(x, y, tau=1e-7):
    integral_value = compute_second_derivative_integral(x, y)
    if np.isnan(integral_value):
        return np.nan
    K = np.sqrt(integral_value / (np.sqrt(120) * tau))
    return int(np.ceil(K))

# Compute the cumulative distribution function F(e)
def compute_cumulative_distribution(x, y):
    y_savgol_second_derivative = savgol_filter(y, window_length=min(31, len(y) - 1), polyorder=3, deriv=2)
    # Compute the denominator integral
    denominator, _ = quad(lambda eps: np.interp(eps, x, np.abs(y_savgol_second_derivative) ** (2/5)), np.min(x), np.max(x))
    
    # Compute the cumulative distribution F(ε)
    F_epsilon = np.zeros_like(x)
    for i, eps in enumerate(x):
        numerator, _ = quad(lambda e: np.interp(e, x, np.abs(y_savgol_second_derivative) ** (2/5)), np.min(x), eps)
        F_epsilon[i] = numerator / denominator
    
    return F_epsilon

# Compute breakpoints
def compute_breakpoints(x, y, K):
    if np.isnan(K) or K <= 1:
        return []
    F_epsilon = compute_cumulative_distribution(x, y)
    # Construct interpolation function for F(e)
    F_interp = interp1d(F_epsilon, x, kind="linear", bounds_error=False, fill_value=(x.min(), x.max()))
    # Compute uniformly divided breakpoints
    breakpoints = F_interp(np.linspace(0, 1, K + 1)[1:-1])  # Remove 0 and 1
    # Insert 0 into correct position to ensure strictly increasing breakpoints
    breakpoints = np.sort(np.append(breakpoints, 0))
    return breakpoints

# Process data from all 24 hours
def process_all_hours(load_variation_dir, hour_dir, output_dir, output_csv):
    all_x = []
    all_y = []
    # Read and merge data from all hours
    for hour in range(1, 25):
        # Load load_variation file, read only the first column
        load_variation_file = os.path.join(load_variation_dir, f'load_variation_divided_{hour}.csv')
        if not os.path.exists(load_variation_file):
            print(f"File {load_variation_file} does not exist, skipping.")
            continue
        load_variation_data = pd.read_csv(load_variation_file, usecols=[0])
        x = np.array(concatenate_columns(load_variation_data))
        
        # Load cost_differences file, read only the first column
        cost_differences_file = os.path.join(hour_dir, str(hour), 'divided_costs.csv')
        if not os.path.exists(cost_differences_file):
            print(f"File {cost_differences_file} does not exist, skipping.")
            continue
        cost_differences_data = pd.read_csv(cost_differences_file, usecols=[0])
        y = np.array(concatenate_columns(cost_differences_data))
        
        # Merge data
        all_x.extend(x)
        all_y.extend(y)
    
    # Sort x and y and remove duplicates
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    sort_indices = np.argsort(all_x)
    x_sorted = all_x[sort_indices]
    y_sorted = all_y[sort_indices]
    x_sorted, unique_indices = np.unique(x_sorted, return_index=True)
    y_sorted = y_sorted[unique_indices]
    
    # Compute smoothed curve using Savitzky-Golay filter
    y_savgol_smooth = savgol_filter(y_sorted, window_length=min(31, len(y_sorted) - 1), polyorder=3)
    
    # Compute optimal number of segments K
    if len(x_sorted) > 3:
        K_opt = compute_optimal_segments(x_sorted, y_savgol_smooth, tau=1e-7)
    else:
        K_opt = np.nan
    
    # Compute breakpoint positions
    breakpoints = compute_breakpoints(x_sorted, y_savgol_smooth, K_opt)
    
    # # Plot cumulative distribution function F(ε)
    # F_epsilon = compute_cumulative_distribution(x_sorted, y_savgol_smooth)
    # plt.figure(figsize=(10, 6))
    # plt.plot(x_sorted, F_epsilon, color='blue', label='$F(\\epsilon)$')
    # for bp in breakpoints:
    #     plt.axvline(x=bp, color='green', linestyle='--')  # Draw breakpoints
    # plt.title('Cumulative Distribution Function (All Hours)')
    # plt.xlabel('$\\epsilon$')
    # plt.ylabel('$F(\\epsilon)$')
    # plt.legend()
    # plt.grid(True)
    # plt.savefig(os.path.join(output_dir, 'all_hours_cumulative_distribution.png'))
    # plt.close()
    
    # # Plot smoothed curve and mark breakpoints
    # plt.figure(figsize=(10, 6))
    # plt.plot(x_sorted, y_savgol_smooth, color='red', label='Smoothing spline')
    # plt.scatter(breakpoints, np.interp(breakpoints, x_sorted, y_savgol_smooth), color='black', marker='D', label='Breakpoints')
    # plt.title('Smoothing Spline and Breakpoints (All Hours)')
    # plt.xlabel('$\\epsilon$')
    # plt.ylabel('$S(\\epsilon)$')
    # plt.legend()
    # plt.grid(True)
    # plt.savefig(os.path.join(output_dir, 'all_hours_smooth_spline_with_breakpoints.png'))
    # plt.close()
    
    # Save global breakpoints and Optimal_Segments_K
    results_df = pd.DataFrame({'Optimal_Segments_K': [K_opt], 'Breakpoints': [breakpoints]})
    results_df.to_csv(output_csv, index=False)
    print(f"Global breakpoints calculation complete. Results saved to {output_csv}")

# Define paths
load_variation_dir = './load_variation'
hour_dir = './hour'
output_dir = './breakpoints_plots'
output_csv = './all_hours_optimal_segments_breakpoints.csv'

# Run the process
process_all_hours(load_variation_dir, hour_dir, output_dir, output_csv)


C:\Users\85033\AppData\Local\Temp\ipykernel_28896\4273346668.py:22: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, _ = quad(lambda eps: np.interp(eps, x, np.abs(y_savgol_second_derivative) ** (2/5)), np.min(x), np.max(x))
C:\Users\85033\AppData\Local\Temp\ipykernel_28896\4273346668.py:37: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, disco

Global breakpoints calculation complete. Results saved to ./all_hours_optimal_segments_breakpoints.csv


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Linear fitting function
def fit_linear_segment(x, y):
    """ Perform linear fitting on a single segment, return slope and intercept """
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    return model.coef_[0], model.intercept_

# Constrained linear fitting function
def fit_constrained_segment(x, y, split_point, y_constraint):
    """ Perform constrained linear fitting on a single segment, forcing the line to pass through (split_point, y_constraint) """
    x_centered = x - split_point
    slope = np.sum(x_centered * (y - y_constraint)) / np.sum(x_centered ** 2)
    intercept = y_constraint - slope * split_point
    return slope, intercept

# Process data from all hours and perform global piecewise linear fitting
def process_all_hours(load_variation_dir, hour_dir, output_dir, output_csv):
    """ Read data -> Merge datasets -> Perform global piecewise linear fitting -> Save plots and fitting parameters """
    all_x = []
    all_y = []
    
    # Read and merge data from all hours
    for hour in range(1, 25):
        # Load load_variation file, read only the first column
        load_variation_file = os.path.join(load_variation_dir, f'load_variation_divided_{hour}.csv')
        if not os.path.exists(load_variation_file):
            print(f"File {load_variation_file} does not exist, skipping Hour {hour}.")
            continue
        
        # Load cost_differences file, read only the first column
        cost_differences_file = os.path.join(hour_dir, str(hour), 'divided_costs.csv')
        if not os.path.exists(cost_differences_file):
            print(f"File {cost_differences_file} does not exist, skipping Hour {hour}.")
            continue
        
        # Read only the first column
        load_variation_data = pd.read_csv(load_variation_file, usecols=[0])
        cost_differences_data = pd.read_csv(cost_differences_file, usecols=[0])
        
        # Merge data
        all_x.extend(load_variation_data.values.flatten())
        all_y.extend(cost_differences_data.values.flatten())
    
    # Convert to NumPy arrays and sort
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    sort_indices = np.argsort(all_x)
    x_sorted = all_x[sort_indices]
    y_sorted = all_y[sort_indices]
    
    # Read global breakpoints
    breakpoints_df = pd.read_csv('./all_hours_optimal_segments_breakpoints.csv')
    if breakpoints_df.empty or 'Breakpoints' not in breakpoints_df.columns:
        print("Global breakpoints not found, skipping.")
        return
    
    # Parse breakpoints
    breakpoints_str = breakpoints_df['Breakpoints'].values[0]
    try:
        if isinstance(breakpoints_str, str):
            breakpoints_str = breakpoints_str.strip('[]').replace('\n', ' ').strip()
            breakpoints = list(map(float, breakpoints_str.split()))
        else:
            breakpoints = list(breakpoints_str)
    except Exception as e:
        print(f"Failed to parse Breakpoints: {e}")
        return
    
    # Add boundaries and ensure first breakpoint is -inf and last is inf
    breakpoints = [-np.inf] + breakpoints + [np.inf]
    
    # Store fitting parameters
    segment_results = []
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(x_sorted, y_sorted, color='blue', label='Original Data', alpha=0.5)
    
    # Iterate through segments and fit (ensure segments are continuous)
    prev_y = None
    for i in range(len(breakpoints) - 1):
        x_segment = x_sorted[(x_sorted > breakpoints[i]) & (x_sorted <= breakpoints[i + 1])]
        y_segment = y_sorted[(x_sorted > breakpoints[i]) & (x_sorted <= breakpoints[i + 1])]
        if len(x_segment) > 1:  # Need at least 2 points to fit
            if prev_y is not None:
                slope, intercept = fit_constrained_segment(x_segment, y_segment, breakpoints[i], prev_y)
            else:
                slope, intercept = fit_linear_segment(x_segment, y_segment)
            # Store fitting results
            segment_results.append([i + 1, breakpoints[i], breakpoints[i + 1], slope, intercept])
            # Plot segment line
            x_fit = np.linspace(breakpoints[i], breakpoints[i + 1], 100)
            y_fit = slope * x_fit + intercept
            plt.plot(x_fit, y_fit, label=f'Segment {i+1}', linewidth=2)
            # Update prev_y for continuity
            prev_y = slope * breakpoints[i + 1] + intercept
        else:
            print(f"Insufficient points at breakpoint {breakpoints[i + 1]}, skipping segment.")
    
    # Plot settings
    plt.title('Global Piecewise Linear Fit (Continuous)')
    plt.xlabel('Load Deviation')
    plt.ylabel('Cost Deviation')
    plt.legend()
    plt.grid(True)
    
    # Save plot
    os.makedirs(output_dir, exist_ok=True)
    plot_file = os.path.join(output_dir, 'global_piecewise_fit_continuous.png')
    plt.savefig(plot_file)
    plt.close()
    print(f"Plot saved to {plot_file}")
    
    # Ensure first breakpoint is -inf and last is inf
    if segment_results[0][1] != -np.inf:
        segment_results[0][1] = -np.inf
    if segment_results[-1][2] != np.inf:
        segment_results[-1][2] = np.inf
    
    # Save results to CSV
    results_df = pd.DataFrame(segment_results, columns=['Segment', 'Breakpoint_Start', 'Breakpoint_End', 'Slope', 'Intercept'])
    results_df.to_csv(output_csv, index=False)
    print(f"Global piecewise linear fitting results saved to {output_csv}")

# Define paths
load_variation_dir = './load_variation'
hour_dir = './hour'
output_dir = './global_piecewise_linear_plots_continuous'
output_csv = './global_piecewise_linear_fit_results_continuous.csv'

# Run global piecewise linear fitting
process_all_hours(load_variation_dir, hour_dir, output_dir, output_csv)


Plot saved to ./global_piecewise_linear_plots_continuous\global_piecewise_fit_continuous.png
Global piecewise linear fitting results saved to ./global_piecewise_linear_fit_results_continuous.csv


C:\Users\85033\AppData\Roaming\Python\Python39\site-packages\numpy\core\function_base.py:151: RuntimeWarning: invalid value encountered in multiply
  y *= step
C:\Users\85033\AppData\Roaming\Python\Python39\site-packages\numpy\core\function_base.py:161: RuntimeWarning: invalid value encountered in add
  y += start


In [3]:
import numpy as np
import pandas as pd

def smooth_segment(breakpoint, slope_left, intercept_left, slope_right, intercept_right, delta=0.0001):
    """
    Perform smoothing at the breakpoint using a quadratic curve approximation.
    :param breakpoint: The location of the breakpoint
    :param slope_left: Slope of the linear segment on the left
    :param intercept_left: Intercept of the linear segment on the left
    :param slope_right: Slope of the linear segment on the right
    :param intercept_right: Intercept of the linear segment on the right
    :param delta: Range around the breakpoint to apply smoothing
    :return: Coefficients a, b, c of the quadratic curve
    """
    # Positions on the left and right of the breakpoint
    x_left = breakpoint - delta
    x_right = breakpoint + delta
    
    # Value and derivative of the left linear segment at x_left
    y_left = slope_left * x_left + intercept_left
    dy_left = slope_left
    
    # Value and derivative of the right linear segment at x_right
    y_right = slope_right * x_right + intercept_right
    dy_right = slope_right
    
    # Construct equations to solve for quadratic coefficients
    A = np.array([[x_left**2, x_left, 1],
                  [x_right**2, x_right, 1],
                  [2 * x_left, 1, 0],
                  [2 * x_right, 1, 0]])
    b = np.array([y_left, y_right, dy_left, dy_right])
    
    # Solve for quadratic coefficients
    a, b, c = np.linalg.lstsq(A, b, rcond=None)[0]
    
    # Validate that the curve meets the boundary conditions
    y_left_curve = a * x_left**2 + b * x_left + c
    y_right_curve = a * x_right**2 + b * x_right + c
    dy_left_curve = 2 * a * x_left + b
    dy_right_curve = 2 * a * x_right + b
    
    assert np.isclose(y_left_curve, y_left), "Left value condition not satisfied"
    assert np.isclose(y_right_curve, y_right), "Right value condition not satisfied"
    assert np.isclose(dy_left_curve, dy_left), "Left derivative condition not satisfied"
    assert np.isclose(dy_right_curve, dy_right), "Right derivative condition not satisfied"
    
    return a, b, c

def smooth_breakpoints(results_df, delta=0.0001):
    """
    Apply smoothing to all breakpoints and store the results in a new CSV file.
    :param results_df: Original piecewise linear fitting results
    :param delta: Range around the breakpoint to apply smoothing
    """
    smoothed_results = []
    
    for i in range(len(results_df) - 1):
        segment_left = results_df.iloc[i]
        segment_right = results_df.iloc[i + 1]
        
        # Breakpoint position
        breakpoint_left = segment_left['Breakpoint_End']
        breakpoint_right = segment_right['Breakpoint_Start']
        
        # If breakpoints match, apply smoothing
        if breakpoint_left == breakpoint_right and not np.isinf(breakpoint_left):
            # Get slopes and intercepts from both sides
            slope_left = segment_left['Slope']
            intercept_left = segment_left['Intercept']
            slope_right = segment_right['Slope']
            intercept_right = segment_right['Intercept']
            
            # Perform smoothing
            a, b, c = smooth_segment(breakpoint_left, slope_left, intercept_left, slope_right, intercept_right, delta)
            
            # Store smoothed result
            smoothed_results.append([breakpoint_left, a, b, c])
    
    # Save smoothed results to a new CSV file
    smoothed_df = pd.DataFrame(smoothed_results, columns=['Breakpoint', 'A', 'B', 'C'])
    smoothed_df.to_csv('./smooth_breakpoints_results.csv', index=False)
    print("Smoothing results have been saved to smooth_breakpoints_results.csv")

# Read original piecewise linear fitting results
results_df = pd.read_csv('./global_piecewise_linear_fit_results_continuous.csv')

# Perform smoothing
smooth_breakpoints(results_df)


Smoothing results have been saved to smooth_breakpoints_results.csv


In [4]:
import numpy as np
import pandas as pd
import os

# Define paths
fit_results_path = './global_piecewise_linear_fit_results_continuous.csv'  # Piecewise linear fitting results
smooth_results_path = './smooth_breakpoints_results.csv'  # Smoothing results
output_dir = './predictions'  # Output folder for prediction results
model_params_dir = './model_params'  # Folder to save model parameters
epsilon_dir = './epsilon_values'  # Folder to save epsilon values during iterations

# Create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(model_params_dir, exist_ok=True)
os.makedirs(epsilon_dir, exist_ok=True)

# Read fitting and smoothing results
fit_results_df = pd.read_csv(fit_results_path)
smooth_results_df = pd.read_csv(smooth_results_path)

# Load training and testing data
X_train = pd.read_csv('./X_train.csv').values
y_train = pd.read_csv('./Y_train.csv').values.reshape(-1, 1)
X_test = pd.read_csv('./X_test.csv').values
y_test = pd.read_csv('./Y_test.csv').values.reshape(-1, 1)

# Apply Min-Max normalization to X in range [-1, 1]
X_train_min = np.min(X_train, axis=0)
X_train_max = np.max(X_train, axis=0)
X_train = 2 * (X_train - X_train_min) / (X_train_max - X_train_min) - 1
X_test = 2 * (X_test - X_train_min) / (X_train_max - X_train_min) - 1

# Add a bias term (all ones) as the last column of X_train and X_test
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

# Hyperparameters
delta = 0.0001  # Piecewise threshold
gamma = 0.001  # Learning rate decay
max_iter = 1000  # Maximum number of iterations
eps = 0  # To avoid division by zero


# Compute piecewise gradient
def piecewise_gradient(epsilon_i, delta, a_k, breakpoints, smooth_results_df):
    """ Compute gradient for sample i """
    grad = 0
    segment_index = np.digitize([epsilon_i], breakpoints)[0] - 1  # Determine which segment the point falls into
    segment_index = int(np.clip(segment_index, 0, len(a_k) - 1))  # Ensure index is within bounds

    # Check if point is near the breakpoint
    if np.abs(epsilon_i - breakpoints[segment_index]) < delta:
        # Query smoothing parameters for the quadratic function
        smooth_params = smooth_results_df[smooth_results_df['Breakpoint'] == breakpoints[segment_index]]
        if not smooth_params.empty:
            a_quad = smooth_params['A'].values[0]
            b_quad = smooth_params['B'].values[0]
            # Compute gradient from quadratic function
            grad = 2 * a_quad * epsilon_i + b_quad
    else:
        # Within a normal linear segment, use slope a_k[segment_index]
        grad = a_k[segment_index]
    return grad


# Evaluation metrics
def evaluate(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))
    return mse, rmse, mae


# Training and prediction function
def train_and_predict():
    """ Train and predict on the entire dataset """
    a_k = fit_results_df['Slope'].values  # Slopes a_k
    breakpoints = fit_results_df['Breakpoint_Start'].values  # Breakpoints

    # Initialize parameters w
    n, d = X_train.shape
    w = np.zeros((d, 1))
    eta = 6  # Initial learning rate

    # Training loop
    for t in range(1, max_iter + 1):
        grad_w = np.zeros_like(w)
        for i in range(n):  # Loop over all samples
            epsilon_i = (X_train[i] @ w - y_train[i]) / y_train[i]
            grad_L_i = piecewise_gradient(epsilon_i, delta, a_k, breakpoints, smooth_results_df)
            grad_w += (grad_L_i / y_train[i]) * X_train[i].reshape(-1, 1)
        w -= eta * grad_w  # Gradient update
        eta = eta / (1 + gamma * t)  # Learning rate decay

        # Print progress every 100 iterations
        if t % 100 == 0:
            y_pred_train = X_train @ w
            mse_train, _, _ = evaluate(y_train, y_pred_train)
            print(f"Iteration {t}: Training MSE={mse_train:.4f}")
            np.save(os.path.join(model_params_dir, f'w_iter_{t}.npy'), w)

    # Predictions on training and test sets
    y_pred_train = X_train @ w
    y_pred_test = X_test @ w

    # Compute final evaluation metrics
    mse_train, rmse_train, mae_train = evaluate(y_train, y_pred_train)
    mse_test, rmse_test, mae_test = evaluate(y_test, y_pred_test)
    print(f"Final Training Set: MSE={mse_train:.4f}, RMSE={rmse_train:.4f}, MAE={mae_train:.4f}")
    print(f"Final Test Set: MSE={mse_test:.4f}, RMSE={rmse_test:.4f}, MAE={mae_test:.4f}")

    # Save prediction results
    pd.DataFrame({"y_true": y_train.flatten(), "y_pred": y_pred_train.flatten()}).to_csv(
        os.path.join(output_dir, 'predictions_train.csv'), index=False)
    pd.DataFrame({"y_true": y_test.flatten(), "y_pred": y_pred_test.flatten()}).to_csv(
        os.path.join(output_dir, 'prediction_GB.csv'), index=False)

    # Save final model parameters
    np.save(os.path.join(model_params_dir, 'w_final.npy'), w)


# Run training and prediction
train_and_predict()


Iteration 100: Training MSE=92908.1724
Iteration 200: Training MSE=92873.9988
Iteration 300: Training MSE=92873.9988
Iteration 400: Training MSE=92873.9988
Iteration 500: Training MSE=92873.9988
Iteration 600: Training MSE=92873.9988
Iteration 700: Training MSE=92873.9988
Iteration 800: Training MSE=92873.9988
Iteration 900: Training MSE=92873.9988
Iteration 1000: Training MSE=92873.9988
Final Training Set: MSE=92873.9988, RMSE=304.7524, MAE=247.7969
Final Test Set: MSE=135356.5933, RMSE=367.9084, MAE=302.9241
